<a href="https://colab.research.google.com/github/daniel-hain/bibliometrics_EIST_2021/blob/master/python/BERTopic_EIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTopic EIST 

* First tryout on EIST topic modelling using BERT an reprentation learning.


## Setup

In [ ]:
# Installing BERTopic
!pip install bertopic

## Data

* Title + Abstracts of all EIST articles up to 02.2023

In [2]:
import pandas as pd

In [3]:
docs = pd.read_csv("https://raw.githubusercontent.com/daniel-hain/bibliometrics_EIST_2021/master/data/data_text.csv")

In [4]:
docs.head()

,UT,PY,text
0,2-S2.0-85117715938,2021,common-pool resources and governance in sustai...
1,2-S2.0-85137288685,2022,introduction to ‘markets in sustainability tra...
2,2-S2.0-85128534377,2022,policy instruments for green-growth of cluster...
3,2-S2.0-85130369331,2022,the transition of agriculture to low carbon pa...
4,2-S2.0-85126615038,2022,hard coal phase-out and the labour market tran...


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

* We start by instantiating BERTopic. 
*We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 
* We will also calculate the topic probabilities. 


In [5]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import KeyBERTInspired
from bertopic.representation import TextGeneration

from sentence_transformers import SentenceTransformer
from transformers.pipelines import pipeline

from hdbscan import HDBSCAN
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer

In [70]:
# Own stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

new_stopwords = ["study", "studies", "paper", "insight", "insights", "research"]
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
# Some hyperparameters
umap_min_dist = 0.1
hdbscan_min_samples = 5

# EMBEDDING MODEL:
embedding_model = "allenai-specter"

# DIMENSIONALITY REDUCTION
umap_model = UMAP(n_neighbors=5, 
                  n_components=5, 
                  min_dist=umap_min_dist, 
                  metric='cosine', 
                  random_state=1337)

# CLUSTERING: resuce min_cluster_size and min_samples
hdbscan_model = HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True, 
                        min_samples=hdbscan_min_samples)

# VECTORIZER: custom vectorizer to get rid of stopwords
vectorizer_model = CountVectorizer(stop_words=stpwrd,
                                   min_df = 10)

# TFIDF Weighting: Weighting model with redued stoppwords
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True,
                                     reduce_frequent_words=True)

In [ ]:
#!pip install openai
#import openai
#from bertopic.representation import OpenAI

#!pip install cohere # if you use cohere
#import cohere
# from bertopic.representation import Cohere

In [12]:
from getpass import getpass
# get from: https://docs.google.com/spreadsheets/d/1Zjy7IFRqowBsB1W7vPRQRYcqANOo82LJEvNR22iAsA8/edit#gid=0

#openai.api_key = getpass('Enter OpenAi secret value: ')
#cohere_api_key = getpass('Enter Cohere secret value: ')

Enter OpenAi secret value: ··········


In [71]:
## REPRESENTATION MODEL: 
representation_model = MaximalMarginalRelevance(diversity=0.5)
#representation_model = KeyBERTInspired()
#representation_model = OpenAI()

In [72]:
# BEWRTopic: specify all custom models and n_grams
topic_model = BERTopic(language="english", 
                       calculate_probabilities=True,
                       verbose=True, 
                       n_gram_range=(1, 3),
                       #
                       embedding_model=embedding_model, 
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model, 
                       vectorizer_model=vectorizer_model,
                       ctfidf_model=ctfidf_model,
                       representation_model=representation_model)

In [73]:
topics, probs = topic_model.fit_transform(docs.loc[:,'text'])

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-03-07 13:51:42,154 - BERTopic - Transformed documents to Embeddings
2023-03-07 13:51:44,312 - BERTopic - Reduced dimensionality
2023-03-07 13:51:44,369 - BERTopic - Clustered reduced embeddings


In [74]:
embeddings = topic_model.embedding_model.embed_documents(docs.loc[:,'text'])

In [ ]:
pd.DataFrame(embeddings).to_csv('embeddings_specter.csv')

In [75]:
# Calculate Hirarchy
hierarchical_topics = topic_model.hierarchical_topics(docs.loc[:,'text'])

100%|██████████| 17/17 [00:01<00:00, 16.03it/s]


In [76]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs.loc[:,'text'], calculate_tokens=True)

100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


In [77]:
# Calculate topics over time
topics_over_time = topic_model.topics_over_time(docs.loc[:,'text'],
                                                docs.loc[:,'PY'])

13it [00:07,  1.72it/s]


## Extracting Topics
* After fitting our model, we can start by looking at the results. 
* Typically, we look at the most frequent topics first as they best represent the collection of documents. 
* -1 refers to all outliers and should typically be ignored. 

In [78]:
freq = topic_model.get_topic_info(); freq.head(50)

,Topic,Count,Name
0,-1,118,-1_learning_electricity_sector_emissions
1,0,235,0_research_socio_practices_insights
2,1,47,1_business_industry_policies_companies
3,2,23,2_renewable_european_creation_countries
4,3,18,3_regime_legitimacy_public_changes
5,4,18,4_deployment_renewable_electricity_political
6,5,17,5_market_network_empirically_government
7,6,17,6_solar_technological_decarbonisation_firms
8,7,16,7_capabilities_actor_systemic_innovation
9,8,14,8_environmental_innovations_diffusion_countries


## Attributes

* There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [79]:
topic_model.topics_[:10]

[0, 5, -1, -1, -1, 13, 0, 0, 8, 0]

# **Visualization**
* There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. 
* Topic modeling is, to a certain extent, quite subjective. * Visualizations help understand the topics that were created. 

## General topic relation

In [80]:
topic_model.visualize_topics()

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [81]:
topic_model.visualize_barchart(top_n_topics=50, n_words = 15)

In [82]:
topic_model.visualize_documents(docs.loc[:,'text'], embeddings=embeddings)

## Visualize Topic Hierarchy

* The topics that were created can be hierarchically reduced. 
* In order to understand the potential hierarchical structure of the topics, we can create clusters and visualize how they relate to one another. 
* This might also help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [83]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [84]:
# Run the visualization with the original embeddings
topic_model.visualize_hierarchical_documents(docs.loc[:,'text'], hierarchical_topics, embeddings=embeddings)

In [ ]:
# Reduce topics if wanted
# topic_model.reduce_topics(docs, nr_topics=30)

## Topics over time

In [85]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [86]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

## Visualize Topic Probabilities

- The variable `probabilities` that is returned from `transform()` or `fit_transform()` can  be used to understand how confident BERTopic is that certain topics can be found in a document. 



In [87]:
# Select a document
doc_nr = 102
docs.loc[doc_nr,'text']

'influences of technological and sectoral contexts on technological innovation systems. this paper analyses how technological and sectoral context structures influence the functional pattern of a focal technological innovation system (tis), focusing on value chain interdependencies. through a case study of the ‘embryonic’ bio-succinate from mixed food waste tis in sweden, we show that all identified context structures exert both negative and positive influences on tis functions by influencing resource availability and accessibility, market conditions and the wider selection environment. contextual influences result from interdependencies throughout the value chain, but in contrast to previous studies, direct structural overlaps are not as relevant as competition for resources, markets and policy support. competition does not only come from the regime but also from other related tiss and sectors. the study also confirms the importance of contextual influences from the international tis.

In [88]:
# Get the probability distribution
topic_model.visualize_distribution(probs[doc_nr], min_probability=0.01)

In [90]:
# Visualize the token-level distributions
doc_topic_dist = topic_model.visualize_approximate_distribution(docs.loc[doc_nr,'text'], topic_token_distr[doc_nr])
doc_topic_dist

,influences,of,technological,and,sectoral,contexts,on,technological,innovation,systems,this,paper,analyses,how,technological,and,sectoral,context,structures,influence,the,functional,pattern,of,focal,technological,innovation,system,tis,focusing,on,value,chain,interdependencies,through,case,study,of,the,embryonic,bio,succinate,from,mixed,food,waste,tis,in,sweden,we,show,that,all,identified,context,structures,exert,both,negative,and,positive,influences,on,tis,functions,by,influencing,resource,availability,and,accessibility,market,conditions,and,the,wider,selection,environment,contextual,influences,result,from,interdependencies,throughout,the,value,chain,but,in,contrast,to,previous,studies,direct,structural,overlaps,are,not,as,relevant,as,competition,for,resources,markets,and,policy,support,competition,does,not,only,come,from,the,regime,but,also,from,other,related,tiss,and,sectors,the,study,also,confirms,the,importance,of,contextual,influences,from,the,international,tis,these,findings,suggest,that,wide,perspective,on,context,structures,and,selection,pressures,should,be,considered,in,future,research
1_business_industry_policies_companies,0.000,0.000,0.000,0.000,0.000,0.000,0.108,0.216,0.216,0.216,0.108,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.102,0.219,0.336,0.336,0.234,0.117,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.100,0.100,0.100,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2_renewable_european_creation_countries,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.102,0.204,0.204,0.204,0.102,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.104,0.209,0.209,0.209,0.104,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3_regime_legitimacy_public_changes,0.000,0.000,0.000,0.000,0.000,0.000,0.125,0.251,0.361,0.465,0.339,0.214,0.104,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.111,0.221,0.221,0.221,0.111,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.138,0.275,0.405,0.535,0.398,0.260,0.130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.